In [1]:
import os
import struct
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt
%matplotlib inline

e:\ProgramData\Miniconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# 1 . Create Dataset -- add random noise pics into MNIST

In [2]:
def load_data(path, kind=''):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

In [3]:
path = './datasets/'

In [4]:
num_classes = 10 

In [5]:
X,y = load_data(path, kind='train')
X_test_,y_test_ = load_data(path, kind='t10k')

In [6]:
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

X_sparse_train = coo_matrix(X)
X, X_sparse_train, y = shuffle(X, X_sparse_train, y, random_state=42)

In [7]:
def keras_style_ds(X_train, X_test, y_train, y_test):
# keras - input
    X_train = X_train / 255
    X_test  = X_test / 255

    X_train = X_train.reshape(len(X_train),1,28,28).astype('float64')
    X_test = X_test.reshape(len(X_test),1,28,28).astype('float64')

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    
    return X_train, X_test, y_train, y_test

# 2 . Build Model -- keras model

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

K.clear_session()

In [9]:
def model_factory():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', 
                     data_format='channels_first',
                     input_shape=(1, 28, 28)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])
    
    return model


In [10]:
# model = model_factory()
# model.summary()

# 3. Training

In [11]:
from sklearn.model_selection import KFold
kf = KFold(num_classes, random_state=None, shuffle=False)

In [12]:
X_mnist_test, y_mnist_test = load_data(path, kind='t10k')
X_mnist_test = X_mnist_test / 255
X_mnist_test = X_mnist_test.reshape(len(X_mnist_test),1,28,28).astype('float64')

y_mnist_test = np_utils.to_categorical(y_mnist_test, num_classes)

In [13]:
# model.fit(X_train, y_train, epochs=12, batch_size=32)
hs_history = []
hs_test_scores = []
for i in range(30
              ):
    
    print("*********** Training for %d time **************" % (i+1))
    print()
    model = model = model_factory()

    for train_index, test_index in kf.split(X):
#         print("TRAIN:", len(train_index), "TEST:", len(test_index))

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train, X_test, y_train, y_test = keras_style_ds(X_train, X_test, y_train, y_test)
        
        history = model.fit(X_train, y_train, epochs=6, batch_size=2048,validation_data = (X_test, y_test),verbose=1)
        hs_history.append(history)  

    
    test_score = model.evaluate(X_mnist_test, y_mnist_test, verbose=1)

    hs_test_scores.append(test_score)
    
    print("Mnist pure test sets -> Loss: %.2f%%" % (test_score[0]*100))
    print("Mnist pure test sets -> Accuracy: %.2f%%" % (test_score[1]*100))
    print()

*********** Training for 1 time **************

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 10s 178us/step - loss: 2.0444 - acc: 0.2805 - val_loss: 1.4889 - val_acc: 0.4670
Epoch 2/6
54000/54000 [==============================] - 6s 120us/step - loss: 1.0648 - acc: 0.6476 - val_loss: 0.4970 - val_acc: 0.8623
Epoch 3/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.5978 - acc: 0.8125 - val_loss: 0.2906 - val_acc: 0.9155
Epoch 4/6
54000/54000 [==============================] - 7s 123us/step - loss: 0.4376 - acc: 0.8674 - val_loss: 0.2052 - val_acc: 0.9360
Epoch 5/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.3317 - acc: 0.9026 - val_loss: 0.1565 - val_acc: 0.9520
Epoch 6/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.2774 - acc: 0.9183 - val_loss: 0.1220 - val_acc: 0.9633
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [========

Epoch 3/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0323 - acc: 0.9906 - val_loss: 0.0124 - val_acc: 0.9960
Epoch 4/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0307 - acc: 0.9908 - val_loss: 0.0135 - val_acc: 0.9950
Epoch 5/6
54000/54000 [==============================] - 6s 119us/step - loss: 0.0297 - acc: 0.9911 - val_loss: 0.0123 - val_acc: 0.9950
Epoch 6/6
10000/10000 [==============================] - 2s 239us/step
Mnist pure test sets -> Loss: 3.14%
Mnist pure test sets -> Accuracy: 99.09%

*********** Training for 2 time **************

Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 7s 127us/step - loss: 1.9527 - acc: 0.3086 - val_loss: 0.9457 - val_acc: 0.7543
Epoch 2/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.9526 - acc: 0.6908 - val_loss: 0.5241 - val_acc: 0.8190
Epoch 3/6
54000/54000 [==============================] - 6s 120us/ste

54000/54000 [==============================] - 7s 122us/step - loss: 0.0353 - acc: 0.9897 - val_loss: 0.0167 - val_acc: 0.9953
Epoch 5/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0341 - acc: 0.9898 - val_loss: 0.0164 - val_acc: 0.9955
Epoch 6/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0338 - acc: 0.9900 - val_loss: 0.0139 - val_acc: 0.9967
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0335 - acc: 0.9900 - val_loss: 0.0157 - val_acc: 0.9952
Epoch 2/6
54000/54000 [==============================] - 7s 120us/step - loss: 0.0338 - acc: 0.9899 - val_loss: 0.0122 - val_acc: 0.9960
Epoch 3/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0328 - acc: 0.9902 - val_loss: 0.0137 - val_acc: 0.9960
Epoch 4/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0294 - acc: 0.9914 - val_loss: 0.0167 - val_acc: 0.9950
Ep

54000/54000 [==============================] - 7s 122us/step - loss: 0.0373 - acc: 0.9884 - val_loss: 0.0185 - val_acc: 0.9947
Epoch 6/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0360 - acc: 0.9898 - val_loss: 0.0262 - val_acc: 0.9915
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0363 - acc: 0.9889 - val_loss: 0.0186 - val_acc: 0.9945
Epoch 2/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0361 - acc: 0.9889 - val_loss: 0.0167 - val_acc: 0.9953
Epoch 3/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0350 - acc: 0.9894 - val_loss: 0.0127 - val_acc: 0.9962
Epoch 4/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0349 - acc: 0.9900 - val_loss: 0.0189 - val_acc: 0.9940
Epoch 5/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0342 - acc: 0.9895 - val_loss: 0.0159 - val_acc: 0.9957
Ep

54000/54000 [==============================] - 7s 121us/step - loss: 0.0472 - acc: 0.9866 - val_loss: 0.0294 - val_acc: 0.9920
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0476 - acc: 0.9867 - val_loss: 0.0150 - val_acc: 0.9957
Epoch 2/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0439 - acc: 0.9873 - val_loss: 0.0318 - val_acc: 0.9907
Epoch 3/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0443 - acc: 0.9875 - val_loss: 0.0220 - val_acc: 0.9933
Epoch 4/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0458 - acc: 0.9869 - val_loss: 0.0180 - val_acc: 0.9943
Epoch 5/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0408 - acc: 0.9882 - val_loss: 0.0183 - val_acc: 0.9937
Epoch 6/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0426 - acc: 0.9878 - val_loss: 0.0184 - val_acc: 0.9943
Tr

...

54000/54000 [==============================] - 7s 120us/step - loss: 0.0420 - acc: 0.9876 - val_loss: 0.0201 - val_acc: 0.9928
Epoch 3/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0397 - acc: 0.9882 - val_loss: 0.0221 - val_acc: 0.9925
Epoch 4/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0371 - acc: 0.9894 - val_loss: 0.0203 - val_acc: 0.9933
Epoch 5/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0386 - acc: 0.9890 - val_loss: 0.0189 - val_acc: 0.9942
Epoch 6/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0366 - acc: 0.9895 - val_loss: 0.0222 - val_acc: 0.9922
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0369 - acc: 0.9895 - val_loss: 0.0141 - val_acc: 0.9962
Epoch 2/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0343 - acc: 0.9899 - val_loss: 0.0204 - val_acc: 0.9942
Ep

54000/54000 [==============================] - 7s 121us/step - loss: 0.0442 - acc: 0.9872 - val_loss: 0.0233 - val_acc: 0.9918
Epoch 4/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0440 - acc: 0.9872 - val_loss: 0.0259 - val_acc: 0.9913
Epoch 5/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0415 - acc: 0.9877 - val_loss: 0.0256 - val_acc: 0.9920
Epoch 6/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0412 - acc: 0.9879 - val_loss: 0.0258 - val_acc: 0.9913
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0423 - acc: 0.9869 - val_loss: 0.0132 - val_acc: 0.9955
Epoch 2/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0395 - acc: 0.9882 - val_loss: 0.0162 - val_acc: 0.9948
Epoch 3/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0377 - acc: 0.9894 - val_loss: 0.0194 - val_acc: 0.9933
Ep

54000/54000 [==============================] - 7s 122us/step - loss: 0.0511 - acc: 0.9856 - val_loss: 0.0259 - val_acc: 0.9915
Epoch 5/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0489 - acc: 0.9862 - val_loss: 0.0307 - val_acc: 0.9905
Epoch 6/6
54000/54000 [==============================] - 7s 120us/step - loss: 0.0485 - acc: 0.9862 - val_loss: 0.0292 - val_acc: 0.9910
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0462 - acc: 0.9865 - val_loss: 0.0295 - val_acc: 0.9910
Epoch 2/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0481 - acc: 0.9865 - val_loss: 0.0319 - val_acc: 0.9918
Epoch 3/6
54000/54000 [==============================] - 7s 123us/step - loss: 0.0428 - acc: 0.9869 - val_loss: 0.0292 - val_acc: 0.9923
Epoch 4/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0419 - acc: 0.9881 - val_loss: 0.0246 - val_acc: 0.9927
Ep

54000/54000 [==============================] - 7s 121us/step - loss: 0.0594 - acc: 0.9824 - val_loss: 0.0373 - val_acc: 0.9903
Epoch 6/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0629 - acc: 0.9822 - val_loss: 0.0437 - val_acc: 0.9877
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 7s 121us/step - loss: 0.0592 - acc: 0.9830 - val_loss: 0.0283 - val_acc: 0.9910
Epoch 2/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0568 - acc: 0.9840 - val_loss: 0.0288 - val_acc: 0.9905
Epoch 3/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0535 - acc: 0.9844 - val_loss: 0.0268 - val_acc: 0.9913
Epoch 4/6
54000/54000 [==============================] - 7s 123us/step - loss: 0.0550 - acc: 0.9847 - val_loss: 0.0235 - val_acc: 0.9918
Epoch 5/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0508 - acc: 0.9858 - val_loss: 0.0227 - val_acc: 0.9932
Ep

54000/54000 [==============================] - 7s 121us/step - loss: 0.0682 - acc: 0.9806 - val_loss: 0.0430 - val_acc: 0.9872
Train on 54000 samples, validate on 6000 samples
Epoch 1/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0662 - acc: 0.9803 - val_loss: 0.0397 - val_acc: 0.9885
Epoch 2/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0622 - acc: 0.9820 - val_loss: 0.0424 - val_acc: 0.9892
Epoch 3/6
54000/54000 [==============================] - 7s 120us/step - loss: 0.0607 - acc: 0.9828 - val_loss: 0.0608 - val_acc: 0.9808
Epoch 4/6
54000/54000 [==============================] - 7s 122us/step - loss: 0.0598 - acc: 0.9829 - val_loss: 0.0390 - val_acc: 0.9892
Epoch 5/6
54000/54000 [==============================] - 6s 120us/step - loss: 0.0575 - acc: 0.9835 - val_loss: 0.0380 - val_acc: 0.9907
Epoch 6/6
54000/54000 [==============================] - 6s 119us/step - loss: 0.0572 - acc: 0.9834 - val_loss: 0.0353 - val_acc: 0.9912
Tr

In [14]:
hs_test_scores

[[0.03142689723534227, 0.9909],
 [0.03870200370835978, 0.9895],
 [0.0344643928633026, 0.9902],
 [0.026529389388191885, 0.9921],
 [0.035385153448789335, 0.9902],
 [0.03395558747232862, 0.9896],
 [0.03400691100120723, 0.9909],
 [0.02929973323619729, 0.9931],
 [0.033941674772367286, 0.9901],
 [0.031093037154682906, 0.9915],
 [0.03085140881552811, 0.9911],
 [0.0316992040579928, 0.9919],
 [0.032467091601995346, 0.9913],
 [0.029870221140215834, 0.992],
 [0.030601308911267097, 0.9914],
 [0.035990498758175, 0.9903],
 [0.0375716170732082, 0.9906],
 [0.039905795964772096, 0.9889],
 [0.027586538915998153, 0.9923],
 [0.03143642617265241, 0.9913],
 [0.032397620336077536, 0.9918],
 [0.027944863242399608, 0.9927],
 [0.03432666587812532, 0.9906],
 [0.032263182342002254, 0.9916],
 [0.028535834266762185, 0.9919],
 [0.03142605495220582, 0.9925],
 [0.031064968930557962, 0.9916],
 [0.030304664982102894, 0.9919],
 [0.040829330381165076, 0.9898],
 [0.03119698236209938, 0.9923]]